<a href="https://colab.research.google.com/github/victoralcantara75/artworks_classifier/blob/master/ProjetoVisao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BEST ARTWORKS OF ALL TIME 
## PROJETO DISCIPLINA - VISÃO COMPUTACIONAL 

##IMPORTAÇÕES 

In [0]:
import os
from sklearn.model_selection import train_test_split
import numpy as np
import cv2
import torch
from torchvision import datasets, models
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn as nn 

In [0]:
!git clone https://github.com/victoralcantara75/best-artworks-of-all-time.git

fatal: destination path 'best-artworks-of-all-time' already exists and is not an empty directory.


In [0]:
epocas = 40
batch_size = 32
learning_rate = 0.001

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

seed = 7
torch.manual_seed(seed)
np.random.seed(seed)

cpu


In [0]:
def diretorio_dados(k):
  
    diretorio = "best-artworks-of-all-time/5-fold/round_"+str(k)+"/"
    
    diretorio_teste = diretorio + "test/"
    diretorio_validacao = diretorio+ "val/"
    diretorio_train = diretorio + "train/"
    
    return  diretorio_train, diretorio_validacao, diretorio_teste

In [0]:
def transformacoes():
  
    transformation = transforms.Compose([ 
                       transforms.Resize(224),
                       transforms.RandomCrop(224),
                       transforms.RandomHorizontalFlip(),
                       transforms.RandomVerticalFlip(),
                       transforms.RandomRotation(15),
                       transforms.ToTensor(),
                       transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                ])

    
    transformation_test = transforms.Compose([ 
                          transforms.Resize(224),
                          transforms.RandomResizedCrop(224),
                          transforms.ToTensor(),
                          transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                ])
    
    return transformation, transformation_test


In [0]:
def carregar_dados():
  
    transf, transf_t = transformacoes()
    
    img_treino = datasets.ImageFolder(dir_treino, transf)
    img_validacao = datasets.ImageFolder(dir_validacao, transf_t)
    img_teste = datasets.ImageFolder(dir_teste, transf_t)
    
    print('Numero de imagens de treino: ', len(img_treino))
    print('Numero de imagens de validacao: ', len(img_validacao))
    print('Numero de imagens de teste: ', len(img_teste))
    
    num_workers=4

    treino = torch.utils.data.DataLoader(img_treino, batch_size=batch_size,num_workers=num_workers, shuffle=True)
    validacao = torch.utils.data.DataLoader(img_validacao, batch_size=batch_size,num_workers=num_workers, shuffle=True)
    teste = torch.utils.data.DataLoader(img_teste, batch_size=batch_size,num_workers=num_workers, shuffle=True)
    
    return treino, validacao, teste

In [0]:
def plot_graph(train_loss, train_accu, test_loss, test_accu, k):
    
    #Index, used to show epoch values
    listaDeIndex = []
    listaDeIndex2 = []

    for i in range(1, len(train_loss)+1):
        listaDeIndex.append(i)

    for i in range(1, len(test_loss)+1):
        listaDeIndex2.append(i)

    accText = ' Gráfico de acurácia em '+str(epocas)+ ' épocas'
    lossText = ' Gráfico de perda(loss) em '+str(epocas)+' épocas'

    plt.title(accText)
    plt.plot(listaDeIndex, train_accu,c='magenta',ls='-',label='Acurácia de treino',fillstyle='none')
    plt.plot(listaDeIndex2, test_accu ,c='green'  ,ls='-',label='Acurácia de validacao' ,fillstyle='none')
    plt.xlabel('Épocas')
    plt.ylabel('Acurácia')
    plt.legend(loc='best')

    grafico_de_acuracia = str("accround_"+str(k)+".png")
    pylab.savefig(grafico_de_acuracia) #Saving
    plt.close() #Para não mostrar essa imagem, pq ela ainda fica salva no plt


    plt.title(lossText)
    plt.plot(listaDeIndex, train_loss,c='magenta',ls='-',label='Erro durante treino',fillstyle='none')
    plt.plot(listaDeIndex2, test_loss ,c='green'  ,ls='-',label='Erro durante validacao' ,fillstyle='none')
    plt.xlabel('Épocas')
    plt.ylabel('Erro')
    plt.legend(loc='best')

    grafico_de_perda = str("lossround_"+str(k)+".png")
    pylab.savefig(grafico_de_perda) #Saving
    plt.close() #Para não mostrar essa imagem, pq ela ainda fica salva no plt


In [0]:
def treinamento(cnn, criterion, optimizer, epocas):

  for epoch in range (1, epocas+1):  # loop over the dataset multiple times
      
      #---------------------- TREINO --------------------
    
      print('\nEpoca {}/{}'.format(epoch, epocas))
    
      cnn.train()
      running_loss = 0.0
      running_corrects = 0
      
      for i, data in enumerate(treino, 0):
          # get the inputs; data is a list of [inputs, labels]
          inputs, labels = data[0].to(device), data[1].to(device)

          # zero the parameter gradients
          optimizer.zero_grad()

          # forward + backward + optimize
          outputs = cnn(inputs)
          _, preds = torch.max(outputs, 1)
          loss = criterion(outputs, labels)
          loss.backward()
          optimizer.step()

          # print statistics
          running_loss += loss.item()
          running_corrects += torch.sum(preds == labels.data)
          
          epoch_loss = running_loss / len(treino.dataset)
          epoch_acc = running_corrects.double() / len(treino.dataset)
          
      print('Train - Loss: {:.4f} Acc: {:.4f}'.format(epoch_loss, epoch_acc))

      #-------------------------- VALIDACAO --------------------
      
      cnn.eval()
      
      running_loss = 0.0
      running_corrects = 0
      
      with torch.no_grad():
         for i, data in enumerate(treino, 0):
          # get the inputs; data is a list of [inputs, labels]
          inputs, labels = data[0].to(device), data[1].to(device)

          # forward + backward + optimize
          outputs = cnn(inputs)
          _, preds = torch.max(outputs, 1)
          loss = criterion(outputs, labels)

          # print statistics
          running_loss += loss.item()
          running_corrects += torch.sum(preds == labels.data)
          
          epoch_loss = running_loss / len(treino.dataset)
          epoch_acc = running_corrects.double() / len(treino.dataset)
          
      print('Valid - Loss: {:.4f} Acc: {:.4f}'.format(epoch_loss, epoch_acc))     
              
  print('Finished Training')
  
  #------------------------- TESTE --------------------
  cnn.eval()
  corrects = 0
  total = 0

  classes = ["Albrecht_Dürer", "Alfred_Sisley", "Amedeo_Modigliani", "Andrei_Rublev", "Andy_Warhol", "Camille_Pissarro", "Caravaggio", "Claude_Monet", "Diego_Rivera", "Diego_Velazquez", "Edgar_Degas", "Edouard_Manet", "Edvard_Munch", "El_Greco", "Eugene_Delacroix", "Francisco_Goya", "Frida_Kahlo", "Georges_Seurat", "Giotto_di_Bondone", "Gustav_Klimt", "Gustave_Courbet", "Henri_de_Toulouse-Lautrec", "Henri_Matisse", "Henri_Rousseau", "Hieronymus_Bosch", "Jackson_Pollock", "Jan_van_Eyck", "Joan_Miro", "Kazimir_Malevich", "Leonardo_da_Vinci", "Marc_Chagall", "Michelangelo", "Mikhail_Vrubel", "Pablo_Picasso", "Paul_Cezanne", "Paul_Gauguin", "Paul_Klee", "Peter_Paul_Rubens", "Pierre-Auguste_Renoir", "Piet_Mondrian", "Pieter_Bruegel", "Raphael", "Rembrandt", "Rene_Magritte", "Salvador_Dali", "Sandro_Botticelli","Titian", "Vasiliy_Kandinskiy", "Vincent_van_Gogh", "William_Turner"]
  class_correct = list(0. for i in range(50))
  class_total = list(0. for i in range(50))

  with torch.no_grad():
      for data in teste:
          images, labels = data[0].to(device), data[1].to(device)
          outputs = cnn(images)
          _, predicted = torch.max(outputs.data, 1)
          total += labels.size(0)
          corrects += (predicted == labels).sum().item()

          running_corrects += (predicted == labels).sum().item()
          correct_tensor = predicted.eq(labels.data.view_as(predicted))    #comparando respostas corretas
          correct = np.squeeze(correct_tensor.numpy()) if not torch.cuda.is_available() else np.squeeze(correct_tensor.cpu().numpy())

          for i in range(len(correct_tensor)):
            label = labels.data[i]
            class_correct[label] += correct[i].item()
            class_total[label] += 1

      for i in range(50):
        if class_total[i] > 0:
            print('Acurácia do teste classe '+classes[i]+': %2d%% (%2d/%2d)' % (100 * class_correct[i] / class_total[i],np.sum(class_correct[i]), np.sum(class_total[i])))
        else:
            print('Acurácia do teste classe %5s: N/A (sem exemplos de treinamento)')

      acc = (100 * corrects / total)

  plot_graph(train_loss_list, train_accu_list, valid_loss_list, valid_accu_list, k)
  print('Acuracia total do round: ', acc)

  return acc

In [0]:


#IMPORTANDO REDE 
cnn = models.resnet50(pretrained=True)

#MUDANDO NUMERO DE CLASSES DE SAIDA
cnn.fc.out_features = 50

# Especificando a loss function (categorical cross-entropy)
criterion = nn.CrossEntropyLoss()
# Definindo o Otimizador
otimizador = optim.SGD(cnn.parameters(), lr=learning_rate)

if torch.cuda.is_available():
  cnn.cuda()
  print(' ')
  

In [0]:
acuracias = []

for k in range (1, 6):

  #CARREGANDO DATASET 
  dir_treino, dir_validacao, dir_teste = diretorio_dados(k)
  treino, validacao, teste = carregar_dados()

  acc = treinamento(cnn, criterion, otimizador, epocas)
  acuracias.append(acc)

resultado = sum(acuracias) / 5

print('\nRESULTADO FINAL : %2d%% ' % (resultado))

Numero de imagens de treino:  4709
Numero de imagens de validacao:  1203
Numero de imagens de teste:  2534

Epoca 1/20


Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 240, in _feed
    send_bytes(obj)
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 240, in _feed
    send_bytes(obj)
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, b

KeyboardInterrupt: ignored